# Self Driving Traffic Sign Detection

Isaiah Jenkins

## Import the required libraries

In [38]:
import tensorflow as tf
from keras import layers, Model, regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
from collections import Counter

import os

## 1. About the data

1. a. Description

Throughout this analysis, we will explore Udacity's Self Driving dataset. This dataset consists of images of thousands of pedestrians, bikers, cars, and traffic lights. Although traffic light images are underrepresented in the dataset, the focus of the analysis will be based solely on traffic light images.

1. b. Data dictionary, 97,942 labels across 11 classes and 15,000 images, 1,720 null examples (images with no labels).

Class Balance across images (labels in each image)

* car - 64,399 - over represented
* pedestrian - 10,806
* trafficLight-Red - 6,870
* trafficLight-Green - 5,465 - underrepresented
* truck - 3,623 - underrepresented
* trafficLight - 2,568 - underrepresented
* biker - 1,864 - under represented
* trafficLight-RedLeft - 1,751 - underrepresented
* trafficLight-GreenLeft - 310 - underrepresented
* trafficLight-Yellow - 272 - underrepresented
* trafficLight-YellowLeft - 14 - underrepresented

## 2. Objectives

Throughout this analysis, we will explore and build various deep learning convolutional neural network (CNN) architectures to detect traffic light signs, aiming to optimize accuracy and efficiency. Our objective is to compare different model variations, such as CNNs with different depths, pre-trained models, and data augmentation techniques, to determine the most effective approach. Potential challenges include handling variations in lighting conditions, occlusions, and small object sizes, which may impact detection performance. Additionally, dataset imbalances and misclassifications due to similar-looking traffic signs could introduce biases, requiring careful preprocessing and model tuning.

## 3. Data Exploration and Preprocessing Images

* Extract traffic sign & traffic light images from the dataset to help with computational efficiency.
* Resize images to standardize dataset making it computationally efficient making it easier to analyze data.
* Normalize pixel values (0-1 range) to help with generalization.
* Upsample underrepresented yellow traffic lights to improve model accuracy.

### Load in data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!unzip -o -q '/content/gdrive/MyDrive/data.zip' -d '/content/'

Mounted at /content/gdrive


In [3]:
DATASET_PATH = 'data/export'
CSV_PATH = os.path.join(DATASET_PATH, '_annotations.csv')

In [4]:
df = pd.read_csv(CSV_PATH) # load annotations

In [5]:
df = df[df['class'].isin(['trafficLight-Red', 'trafficLight-Yellow', 'trafficLight-Green'])]

In [6]:
# 🔹 Count Class Distribution
print("Original Class Distribution:\n", df["class"].value_counts())

Original Class Distribution:
 class
trafficLight-Red       13673
trafficLight-Green     10838
trafficLight-Yellow      541
Name: count, dtype: int64


In [7]:
# Separate Classes
df_red = df[df["class"] == "trafficLight-Red"]
df_yellow = df[df["class"] == "trafficLight-Yellow"]
df_green = df[df["class"] == "trafficLight-Green"]

In [8]:
# Upsample Yellow traffic light images
max_class_size = max(len(df_red), len(df_green))  # Find largest class count
df_yellow_upsampled = df_yellow.sample(n=max_class_size, replace=True, random_state=42)

In [9]:
# 🔹 Combine the Balanced Dataset
df_balanced = pd.concat([df_red, df_yellow_upsampled, df_green], ignore_index=True)
# 🔹 Shuffle Dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [10]:
# 🔹 Count Class Balanced Distribution
print("Balanced Class Distribution:\n", df_balanced["class"].value_counts())

Balanced Class Distribution:
 class
trafficLight-Red       13673
trafficLight-Yellow    13673
trafficLight-Green     10838
Name: count, dtype: int64


In [11]:
file_names = df_balanced['filename'].values # image files names
labels = df_balanced['class'].values # classes

In [12]:
print('file_names', file_names.shape, 'labels', labels.shape)

file_names (38184,) labels (38184,)


In [13]:
label_map = {"trafficLight-Red": 0, "trafficLight-Yellow": 1, "trafficLight-Green": 2, }  # Adjust based on dataset
labels = [label_map[label] for label in labels if label in label_map]  # Convert text labels to integers
labels = to_categorical(labels, num_classes=3)  # Convert to one-hot encoding

In [14]:
# Split Data into Train (80%) and Test (20%)
train_files, test_files, train_labels, test_labels = train_test_split(
    file_names, labels, test_size=0.2, random_state=42, stratify=labels, shuffle=True
)

In [15]:
IMG_SIZE = (224, 224)

In [16]:
# Function to Load & Preprocess Images
def load_and_preprocess_image(file_name, label):
    img_path = os.path.join(DATASET_PATH, file_name)  # Update image folder path
    img = load_img(img_path, target_size=IMG_SIZE)  # Load & Resize Image
    img = img_to_array(img) / 255.0  # Convert to NumPy array & Normalize (0-1)
    return img, label

In [17]:
# Create Generators for Train & Test Data
def data_generator(file_list, label_list):
    for f, l in zip(file_list, label_list):
        yield load_and_preprocess_image(f, l)  # Load one image at a time

In [18]:
BATCH_SIZE = 32

In [19]:
# Create a dataset that loads images on demand
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_files, train_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).shuffle(1000).batch(BATCH_SIZE).take(32).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_files, test_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
# Initialize a counter
class_counts = Counter()

# Iterate over the buffered dataset
for images, labels in train_dataset.unbatch().take(1000):  # Unbatch & take 1000 images
    label_index = np.argmax(labels.numpy())  # Convert one-hot to class index
    class_counts[label_index] += 1

# Print class distribution
print("Class Distribution in Buffered 1000 Images:", class_counts)

Class Distribution in Buffered 1000 Images: Counter({0: 366, 1: 363, 2: 271})


## 4. CNN Models (2 models, 3 variations per model)

#### ResNet50 - Variation 1 (With Average Pooling)

In [ ]:
# Layers
num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

In [ ]:
# Compile
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 25,688,963 (98.00 MB)

 Trainable params: 25,635,843 (97.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
# Fit on training set and compare against validation set after each epoch
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
     32/Unknown 11s 280ms/step - accuracy: 0.9494 - loss: 0.1180

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


32/32 ━━━━━━━━━━━━━━━━━━━━ 31s 903ms/step - accuracy: 0.9494 - loss: 0.1187 - val_accuracy: 0.2838 - val_loss: 1.2641
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 898ms/step - accuracy: 0.9490 - loss: 0.1679 - val_accuracy: 0.3581 - val_loss: 1.2924
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 889ms/step - accuracy: 0.9555 - loss: 0.1406 - val_accuracy: 0.3445 - val_loss: 1.1173
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 894ms/step - accuracy: 0.9768 - loss: 0.0834 - val_accuracy: 0.2813 - val_loss: 1.1107
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 893ms/step - accuracy: 0.9702 - loss: 0.0856 - val_accuracy: 0.3581 - val_loss: 1.1138


In [ ]:
# Evaluate - Loss, Accuracy
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.3565 - loss: 1.1149


[1.113835096359253, 0.35812491178512573]

#### RestNet50 - Variation 2 (With Dropout layer & RMSprop optimizer)

In [ ]:
# Layers
num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

In [ ]:
# Compile
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 26,212,227 (99.99 MB)

 Trainable params: 26,159,107 (99.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
# Fit variation
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.3712 - loss: 3.2397 - val_accuracy: 0.2838 - val_loss: 4273.8750
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 884ms/step - accuracy: 0.4783 - loss: 1.1038 - val_accuracy: 0.3581 - val_loss: 33.5013
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 873ms/step - accuracy: 0.5092 - loss: 1.0759 - val_accuracy: 0.3181 - val_loss: 1.1628
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 877ms/step - accuracy: 0.6667 - loss: 0.7676 - val_accuracy: 0.3581 - val_loss: 2.5403
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 884ms/step - accuracy: 0.7252 - loss: 0.6429 - val_accuracy: 0.2838 - val_loss: 1.1255


In [ ]:
# Evaluate - Loss, Accuracy
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.3571 - loss: 2.9629


[2.9884872436523438, 0.355898916721344]

#### ResNet50 - Variation 3 (Reduced model complexity layers, Reverted optimizer to Adam, Dropout layers)

In [ ]:
# Layers
num_classes = 3

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

In [ ]:
# Compile
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 24,638,339 (93.99 MB)

 Trainable params: 24,585,219 (93.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
# Fit variation
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.5237 - loss: 1.5439 - val_accuracy: 0.3581 - val_loss: 1277.7788
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 891ms/step - accuracy: 0.6486 - loss: 0.8713 - val_accuracy: 0.3581 - val_loss: 152.8631
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 882ms/step - accuracy: 0.7562 - loss: 0.6823 - val_accuracy: 0.3581 - val_loss: 1.8637
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 880ms/step - accuracy: 0.8137 - loss: 0.4528 - val_accuracy: 0.3581 - val_loss: 4.1290
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 884ms/step - accuracy: 0.8577 - loss: 0.4078 - val_accuracy: 0.3581 - val_loss: 2.0549


In [ ]:
# Evaluate - Loss, Accuracy
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - accuracy: 0.3629 - loss: 1161.7773


[1168.3563232421875, 0.35812491178512573]

#### Custom CNN - Variation 1 (Shallow CNN, Reducing complexity for small training buffer, Added dropout layer)

In [ ]:
# Layers
num_classes = 3

model = Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,779 (91.20 MB)

 Trainable params: 23,907,779 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Fit variation
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
     32/Unknown 11s 81ms/step - accuracy: 0.3863 - loss: 4.2521

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 665ms/step - accuracy: 0.3888 - loss: 4.1906 - val_accuracy: 0.6361 - val_loss: 0.8286
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 591ms/step - accuracy: 0.7063 - loss: 0.6971 - val_accuracy: 0.7519 - val_loss: 0.5674
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 587ms/step - accuracy: 0.8374 - loss: 0.4103 - val_accuracy: 0.8272 - val_loss: 0.4426
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 583ms/step - accuracy: 0.8975 - loss: 0.3136 - val_accuracy: 0.8438 - val_loss: 0.4049
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 587ms/step - accuracy: 0.9037 - loss: 0.2696 - val_accuracy: 0.8561 - val_loss: 0.3681


In [ ]:
# Evaluate
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 17s 72ms/step - accuracy: 0.8545 - loss: 0.3601


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


[0.36812537908554077, 0.8560953140258789]

#### Custom CNN - Variation 2 (With Batch Normalization)

In [116]:
# Layers
num_classes = 3

model = Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

In [117]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_56 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_56 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_57 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_57 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_28 (Flatten)                 │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_81 (Dense)                     │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_55 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_82 (Dense)                     │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,908,291 (91.20 MB)

 Trainable params: 23,908,035 (91.20 MB)

 Non-trainable params: 256 (1.00 KB)

In [118]:
# Fit variation
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 25s 608ms/step - accuracy: 0.4707 - loss: 1.3523 - val_accuracy: 0.5860 - val_loss: 1.7196
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 587ms/step - accuracy: 0.7484 - loss: 0.6353 - val_accuracy: 0.4741 - val_loss: 3.0063
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 595ms/step - accuracy: 0.8160 - loss: 0.4853 - val_accuracy: 0.4512 - val_loss: 2.4327
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 607ms/step - accuracy: 0.8904 - loss: 0.3254 - val_accuracy: 0.3581 - val_loss: 4.2934
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 589ms/step - accuracy: 0.9057 - loss: 0.2646 - val_accuracy: 0.4697 - val_loss: 1.7756


In [119]:
# Evaluate
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 17s 71ms/step - accuracy: 0.4691 - loss: 1.7444


[1.7756257057189941, 0.4696870446205139]

#### Custom CNN - Variation 3 (Removed batch normalization, Increased first dropout layer by 0.1, Added additional dropout layer of 0.4)

In [112]:
# Layers
num_classes = 3

model = Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.6),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

In [113]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_54 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_54 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_55 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_55 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_27 (Flatten)                 │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_78 (Dense)                     │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_53 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_79 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_54 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_80 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,915,843 (91.23 MB)

 Trainable params: 23,915,843 (91.23 MB)

 Non-trainable params: 0 (0.00 B)

In [114]:
# Fit variation
early_stop = EarlyStopping(monitor='val_accuracy', patience=0, restore_best_weights=True)
model.fit(train_dataset, epochs=10, validation_data=test_dataset, callbacks=[early_stop])

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 25s 628ms/step - accuracy: 0.3658 - loss: 2.6364 - val_accuracy: 0.4727 - val_loss: 1.0460
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 610ms/step - accuracy: 0.4874 - loss: 1.0184 - val_accuracy: 0.6704 - val_loss: 0.7184
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 610ms/step - accuracy: 0.6505 - loss: 0.7790 - val_accuracy: 0.7550 - val_loss: 0.5659
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 610ms/step - accuracy: 0.7433 - loss: 0.6275 - val_accuracy: 0.8148 - val_loss: 0.4740
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 22s 616ms/step - accuracy: 0.8327 - loss: 0.4488 - val_accuracy: 0.8325 - val_loss: 0.4435
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 22s 623ms/step - accuracy: 0.8571 - loss: 0.4033 - val_accuracy: 0.8558 - val_loss: 0.3987
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 598ms/step - accuracy: 0.8854 - loss: 0.3057 - val_accuracy: 0.8514 - val_loss: 0.4038


In [115]:
# Evaluate variation
model.evaluate(test_dataset)

239/239 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - accuracy: 0.8523 - loss: 0.3964


[0.3986527621746063, 0.8558334708213806]

### Summary of Models

Throughout this analysis, two models with three variations each were tested. Three variations of the pre-trained model ResNet50 were trained to utilize the power of transfer learning. To reduce the generalization error, three variations of a custom shallow CNN were tested. All these variations of the ResNet50 model were overfitting because the variation was too complex despite hyperparameters and fine-tuning. Through further research, the ResNet50 model parameters were exceedingly complex on a small training buffer of 1,000 images. As a result, utilizing a custom shallow CNN was better suited for training on this small buffer. From these first two variations, the training accuracy was better than the ResNet50 variations. However, those variations were overfitted by adding a dropout layer and utilizing batch normalization. In the last variation, an additional dropout layer was added and was fine-tuned to reduce the overfitting. In addition, the epochs were increased and an early stopping callback was added to monitor the validation loss accuracy to retain the best weights. In conclusion, this variation did not overfit throughout training the epochs. It became well-balanced towards the end of the fitting with a training accuracy of 85% and a validation accuracy of 85%.

## 5. Insights and Key Findings

When training with images, the first idea was to figure out how to save time and resources by utilizing the earlier layers of a pre-trained neural network, ResNet50. After training three variations of ResNet50, those results were overfitting because the trainable parameters were meant for larger training datasets. The second idea was to train a simpler model with a custom CNN with fewer parameters. After fitting three variations, the last variation was well-balanced. Overall, the lesson from training these models was to start small and then gradually work up to more complexity.

## 6. Next Steps

For the next steps, revisiting the customized CNNs and exploring regularization techniques such as L1 Lasso and L2 Ridge would be worth analysis. Although ResNet50 is meant for larger datasets, exploring other transfer learning models like the pre-trained MobileNet model would be meant for smaller datasets. Incorporating these strategies could enhance model performance and adaptability, ultimately leading to more robust results in our analysis.